In [71]:
import numpy as np

In [72]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [73]:
import csv
import openpyxl
import pandas as pd
from pandas import DataFrame

In [74]:
metadata = pd.read_csv(r"C:\Users\aniru\Desktop\crawling\movies_metadata.csv",low_memory=False)
metadata = pd.DataFrame(metadata)

In [75]:
movies = pd.read_csv(r"C:\Users\aniru\Downloads\ml-25m\movies.csv")
movies = pd.DataFrame(movies)
tags = pd.read_csv(r"C:\Users\aniru\Downloads\ml-25m\tags.csv")
tags = pd.DataFrame(tags)


In [76]:
Keywords = pd.merge(movies,tags,on = 'movieId')

In [77]:
Keywords = Keywords.groupby('movieId')['tag'].apply(list)

In [78]:
metadata = metadata.rename(columns = {'imdb_id':'ID','id':'movieId'})
metadata

,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",27205,1375666.0,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,2010-07-14,825532764,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075
1,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",155,468569.0,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,2008-07-16,1004558444,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269
2,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,499549.0,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114
3,220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",24428,848228.0,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,2012-04-25,1519557910,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000
4,58000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",293660,1431045.0,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,/inVq3FRqcYIRl2la8iZikYYxFNR.jpg,2016-02-09,783112979,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45432,0,[],400796,1560959.0,en,Irwin & Fran 2013,Professor Irwin Corey is now 101 years old and...,0.000000,/eyqXmDv1wGnhKCj3JpOLQfV9ZtB.jpg,NaN,0,83.0,[],Released,Married for over seventy years to his chain sm...,Irwin & Fran 2013,False,0.0,0
45433,0,[],441826,2137792.0,en,Drzace traby,"Natalia Brożyńska's animation debut, about dra...",0.000006,NaN,NaN,0,4.0,[],Released,NaN,Shivering Trunks,False,0.0,0
45434,0,[],398295,5256044.0,pt,Mundo Cão,NaN,0.000331,NaN,NaN,0,0.0,[],Released,NaN,Mundo Cão,False,0.0,0
45435,0,[],439314,NaN,en,The Garden of Afflictions 2017,Brazilian philosopher Olavo de Carvalho's thin...,0.000000,NaN,NaN,0,0.0,[],Released,NaN,The Garden of Afflictions 2017,False,0.0,0


In [79]:
meanvotes = metadata['vote_average'].mean()
print(meanvotes)

5.6179633338463955


In [80]:
Bmark = metadata['vote_count'].quantile(0.90)
print(Bmark)

160.0


In [81]:
Topmovies = metadata.copy().loc[metadata['vote_count'] >= Bmark]

In [82]:
def weighted_rating(x, Bmark=Bmark, meanvotes=meanvotes):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+Bmark) * R) + (Bmark/(Bmark+v) * meanvotes)

In [83]:
Topmovies['score'] = Topmovies.apply(weighted_rating, axis=1)

In [84]:
Topmovies

,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
0,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",27205,1375666.0,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,2010-07-14,825532764,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075,8.072102
1,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",155,468569.0,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,2008-07-16,1004558444,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269,8.265474
2,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,499549.0,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114,7.179377
3,220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",24428,848228.0,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,2012-04-25,1519557910,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000,7.376552
4,58000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",293660,1431045.0,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,/inVq3FRqcYIRl2la8iZikYYxFNR.jpg,2016-02-09,783112979,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444,7.375429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4548,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",7300,117247.0,en,One Fine Day,"Melanie Parker, an architect and mother of Sam...",7.009946,/yJETLNpYeEpDar7tJK1KedYGLgx.jpg,1996-12-20,97529550,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,She was having a perfectly bad day... Then he ...,One Fine Day,False,6.1,160,5.858982
4549,5000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",11569,104409.0,en,Hellraiser III: Hell on Earth,Pinhead is trapped in a sculpture and fortunat...,6.546551,/hTzWX5HYcKpg5DdQIDoFy39yMAm.jpg,1992-09-11,12534961,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What started in hell, will end on Earth",Hellraiser III: Hell on Earth,False,5.3,160,5.458982
4550,1000000,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",10176,89869.0,en,The Quiet Earth,"After a top-secret experiment misfires, a scie...",7.782919,/61OEjyXUPpHZ8aEc9avfBKimLJ9.jpg,1985-09-08,0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The End of the World is Just the Beginning,The Quiet Earth,False,6.7,160,6.158982
4551,1500000,"[{'id': 35, 'name': 'Comedy'}]",10794,63415.0,en,The Party,"Hrundi V. Bakshi, an accident-prone actor from...",5.354787,/pC6FB4fLIVRCpMzIbUf1xOVobmz.jpg,1968-04-04,0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,If you've ever been to a wilder party... you'r...,The Party,False,7.3,160,6.458982


In [85]:
Topmovies = Topmovies.sort_values('score', ascending=False)

In [86]:
Topmovies.head(20)

,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
19,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",278,111161.0,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,51.645403,/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg,1994-09-23,28341469,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358,8.445865
61,6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",238,68646.0,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",41.109264,/rPdtLWNsZmAtoZl9PK7S2wE3qiS.jpg,1972-03-14,245066411,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024,8.425432
1636,13200000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",19404,112870.0,hi,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",34.457024,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,1995-10-20,100000000,190.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Come... Fall In Love,Dilwale Dulhania Le Jayenge,False,9.1,661,8.421406
1,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",155,468569.0,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,2008-07-16,1004558444,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269,8.265474
8,63000000,"[{'id': 18, 'name': 'Drama'}]",550,137523.0,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,1999-10-15,100853753,139.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.3,9678,8.256381
17,8000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",680,110912.0,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",140.950236,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,1994-09-10,213928762,154.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Just because you are a character doesn't mean ...,Pulp Fiction,False,8.3,8670,8.251401
149,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",424,108052.0,en,Schindler's List,The true story of how businessman Oskar Schind...,41.725123,/yPisjyLweCl1tbgwgtzBCNCBle.jpg,1993-11-29,321365567,195.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,"Whoever saves one life, saves the world entire.",Schindler's List,False,8.3,4436,8.206631
152,3300000,"[{'id': 18, 'name': 'Drama'}]",244786,2582802.0,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,2014-10-10,13092000,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,False,8.3,4376,8.205396
180,15000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",129,245429.0,ja,千と千尋の神隠し,A ten year old girl who wanders away from her ...,41.048867,/ynXoOxmDHNQ4UAy0oU6avW71HVW.jpg,2001-07-20,274925095,125.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,The tunnel led Chihiro to a mysterious town...,Spirited Away,False,8.3,3968,8.196045
208,20000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",637,118799.0,it,La vita è bella,A touching story of an Italian book seller of ...,39.394970,/f7DImXDebOs148U4uPjI61iDvaK.jpg,1997-12-20,229400000,116.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"An unforgettable fable that proves love, famil...",Life Is Beautiful,False,8.3,3643,8.187161


In [87]:
Topmovies[Topmovies['original_language']=='hi']


,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
1636,13200000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",19404,112870.0,hi,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",34.457024,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,1995-10-20,100000000,190.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Come... Fall In Love,Dilwale Dulhania Le Jayenge,False,9.1,661,8.421406
1321,9000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",20453,1187043.0,hi,3 Idiots,In the tradition of “Ferris Bueller’s Day Off”...,7.675621,/wbE5SRTZFtQxgj2nIo4HJpQDk0k.jpg,2009-12-23,70000000,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Don't BE A STUPID BE AN I.D.I.O.T.,3 Idiots,False,7.8,850,7.454331
3318,13000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",297222,2338151.0,hi,PK,This experimental satire is a comedy of ideas ...,12.587520,/sYbWZa413QmoBx0quaTLYMZHMAi.jpg,2014-12-18,120000000,153.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Peekay hai hum Peekay,PK,False,7.7,262,6.910602
3545,12000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",26022,1188996.0,hi,My Name Is Khan,"Rizwan Khan, a Muslim from the Borivali sectio...",11.264586,/ipMQOfE4juN3FQh7uL1os1rwYRs.jpg,2010-02-12,42345360,145.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,I am Muslim and i am not a terrorist,My Name Is Khan,False,7.7,237,6.860892
4052,2600000,"[{'id': 18, 'name': 'Drama'}]",7508,986264.0,hi,Taare Zameen Par,Ishaan Awasthi is an eight-year-old whose worl...,6.093304,/t8x4j5KtI6hZRUVtPPztNBo1q5W.jpg,2007-12-21,28430000,165.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Every child is special,Taare Zameen Par,False,7.7,194,6.758966
4443,14000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",348892,3863552.0,hi,Bajrangi Bhaijaan,A young mute girl from Pakistan loses herself ...,6.312240,/fAVCr7rfEKbQzre4ETCFXNDV6U3.jpg,2015-07-17,98000000,163.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}, {'iso_...",Released,NaN,Bajrangi Bhaijaan,False,7.7,165,6.674997
3659,1000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",191714,2350496.0,hi,Dabba,A mistaken delivery in Mumbai's famously effic...,10.766409,/xFJqU1W5WlJiKr4Witnb7h9HNHn.jpg,2013-09-19,4235151,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,The Lunchbox,False,7.3,226,6.602783


In [88]:
Topmovies['overview'].head(20)

19      Framed in the 1940s for the double murder of h...
61      Spanning the years 1945 to 1955, a chronicle o...
1636    Raj is a rich, carefree, happy-go-lucky second...
1       Batman raises the stakes in his war on crime. ...
8       A ticking-time-bomb insomniac and a slippery s...
17      A burger-loving hit man, his philosophical par...
149     The true story of how businessman Oskar Schind...
152     Under the direction of a ruthless instructor, ...
180     A ten year old girl who wanders away from her ...
208     A touching story of an Italian book seller of ...
230     In the continuing saga of the Corleone crime f...
294     While serving time for insanity at a state men...
21      A man with a low IQ has accomplished great thi...
63      The epic saga continues as Luke Skywalker, in ...
399     When larcenous real estate clerk Marion Crane ...
88      A true story of two men who should never have ...
1087    High schoolers Mitsuha and Taki are complete s...
155     Leon, 

In [89]:
Test = metadata.sort_values(by = 'vote_count',ascending = False)
TestData = Test[0:20000]
TestData

,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",27205,1375666.0,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,2010-07-14,825532764,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075
1,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",155,468569.0,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,2008-07-16,1004558444,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269
2,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,499549.0,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114
3,220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",24428,848228.0,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,2012-04-25,1519557910,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000
4,58000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",293660,1431045.0,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,/inVq3FRqcYIRl2la8iZikYYxFNR.jpg,2016-02-09,783112979,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20507,0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",41938,98093.0,en,Physical Evidence,A police officer suspended and now accused of ...,1.138586,/95JrCZlbXrfqRrbUVl6y5zmQNYp.jpg,1989-01-27,0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If he's innocent, the truth could save him. If...",Physical Evidence,False,4.8,12
20508,0,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",52794,94713.0,en,Baxter,A white Bull Terrier named Baxter is given to ...,0.005568,/gbSiQmWfBSirJvnXmqaI1zn4n6L.jpg,1989-01-18,0,82.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,Beware of the dog who thinks.,Baxter,False,6.4,12
20536,3000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",29463,89444.0,en,Krush Groove,"Russell Walker is a young, successful manager ...",1.560562,/ZccQA1MQejTGeqotMnlGOAjs1c.jpg,1985-10-25,0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's chillin'!,Krush Groove,False,7.0,12
20581,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",15227,80772.0,en,Ginga tetsudô Three-Nine,"In the future, one can achieve immortality by ...",1.356131,/TDSbp7OoIQ15eZGzLiJrssIvqI.jpg,1979-08-04,0,128.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Galaxy Express 999,False,6.2,12


In [90]:
tfidf = TfidfVectorizer(stop_words='english')
TestData['overview'] = TestData['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(TestData['overview'])
tfidf_matrix.shape

<ipython-input-90-9921ee808456>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TestData['overview'] = TestData['overview'].fillna('')


(20000, 45633)

In [91]:
tfidf.get_feature_names()

['00',
 '000',
 '000km',
 '000th',
 '007',
 '0093',
 '01',
 '042',
 '05pm',
 '07',
 '07am',
 '088',
 '10',
 '100',
 '1000',
 '1001',
 '100th',
 '101',
 '101st',
 '103',
 '108',
 '1080s',
 '108th',
 '10b',
 '10crores',
 '10mn',
 '10th',
 '10x',
 '11',
 '111',
 '1138',
 '114',
 '117',
 '118',
 '1183',
 '119',
 '11th',
 '12',
 '120',
 '1200',
 '1206',
 '1215',
 '1250',
 '125th',
 '12th',
 '13',
 '130',
 '1300',
 '1303',
 '133',
 '1344',
 '1348',
 '138',
 '13b',
 '13th',
 '14',
 '140',
 '1400',
 '1408',
 '1413',
 '142',
 '1429',
 '143',
 '144',
 '145',
 '1475',
 '148',
 '1492',
 '14pm',
 '14th',
 '15',
 '150',
 '1500',
 '150000',
 '1509',
 '150th',
 '1536',
 '155',
 '1564',
 '1572',
 '1592',
 '1595',
 '1597',
 '1598',
 '15th',
 '15yrs',
 '16',
 '160',
 '1600',
 '1600s',
 '161',
 '1610',
 '1630s',
 '164',
 '1644',
 '1649',
 '165',
 '1671',
 '168',
 '1681',
 '1691',
 '1692',
 '16th',
 '17',
 '170',
 '1700',
 '1700s',
 '174',
 '1740',
 '1760',
 '1776',
 '1777',
 '1787',
 '1791',
 '1793',
 '17

In [92]:
similarity_vector = linear_kernel(tfidf_matrix, tfidf_matrix)

In [93]:
indices = pd.Series(TestData.index, index=TestData['title']).drop_duplicates()

In [94]:
indices[20:30]

title
The Lord of the Rings: The Return of the King    20
Forrest Gump                                     21
Star Wars: The Force Awakens                     22
Titanic                                          23
Skyfall                                          24
Suicide Squad                                    25
The Lord of the Rings: The Two Towers            26
Batman Begins                                    27
Captain America: Civil War                       28
The Martian                                      29
dtype: int64

In [95]:
def get_recommendations(title, similarity_vector=similarity_vector):
    idx = indices[title]
    sim_scores = list(enumerate(similarity_vector[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return TestData['title'].iloc[movie_indices]

In [116]:
get_recommendations('Captain America: Civil War')

15108           Tell Them Willie Boy Is Here
6298                            The Tin Drum
69       Captain America: The Winter Soldier
36                   Avengers: Age of Ultron
6817                   The Last Man on Earth
14776                                 Hatred
6645                               Sidewalls
2736                                   U-571
6830                                  Wolves
9864                            I Smile Back
Name: title, dtype: object

In [97]:
import numpy as np

In [98]:
Keywords = pd.DataFrame(Keywords)
Keywords

,tag
movieId,
1,"[Owned, imdb top 250, Pixar, Pixar, time trave..."
2,"[Robin Williams, time travel, fantasy, based o..."
3,"[funny, best friend, duringcreditsstinger, fis..."
4,"[based on novel or book, chick flick, divorce,..."
5,"[aging, baby, confidence, contraception, daugh..."
...,...
208813,[might like]
208933,"[black and white, deal with the devil]"
209035,"[computer animation, Japan, mass behavior, mas..."


In [99]:

credits = pd.read_csv(r"C:\Users\aniru\Desktop\crawling\credits.csv")
credits = credits.rename(columns = {'id':'movieId'})
credits


,cast,crew,movieId
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [100]:

TestData = pd.merge(TestData,credits, on='movieId')



In [101]:
TestData

,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",27205,1375666.0,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,2010-07-14,...,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075,"[{'cast_id': 1, 'character': 'Dom Cobb', 'cred...","[{'credit_id': '56e8462cc3a368408400354c', 'de..."
1,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",155,468569.0,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,2008-07-16,...,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269,"[{'cast_id': 35, 'character': 'Bruce Wayne / B...","[{'credit_id': '55a0eb4a925141296b0010f8', 'de..."
2,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,499549.0,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,2009-12-10,...,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
3,220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",24428,848228.0,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,2012-04-25,...,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000,"[{'cast_id': 46, 'character': 'Tony Stark / Ir...","[{'credit_id': '52fe4495c3a368484e02b1cf', 'de..."
4,58000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",293660,1431045.0,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,/inVq3FRqcYIRl2la8iZikYYxFNR.jpg,2016-02-09,...,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444,"[{'cast_id': 99, 'character': 'Wade Wilson / D...","[{'credit_id': '57e564fac3a3681beb0000cc', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20034,0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",41938,98093.0,en,Physical Evidence,A police officer suspended and now accused of ...,1.138586,/95JrCZlbXrfqRrbUVl6y5zmQNYp.jpg,1989-01-27,...,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If he's innocent, the truth could save him. If...",Physical Evidence,False,4.8,12,"[{'cast_id': 2, 'character': 'Joe Paris', 'cre...","[{'credit_id': '598dd8ad92514178ac015543', 'de..."
20035,0,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",52794,94713.0,en,Baxter,A white Bull Terrier named Baxter is given to ...,0.005568,/gbSiQmWfBSirJvnXmqaI1zn4n6L.jpg,1989-01-18,...,82.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,Beware of the dog who thinks.,Baxter,False,6.4,12,"[{'cast_id': 1, 'character': 'Madame Deville',...","[{'credit_id': '54359519c3a36842210002dd', 'de..."
20036,3000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",29463,89444.0,en,Krush Groove,"Russell Walker is a young, successful manager ...",1.560562,/ZccQA1MQejTGeqotMnlGOAjs1c.jpg,1985-10-25,...,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's chillin'!,Krush Groove,False,7.0,12,"[{'cast_id': 1, 'character': 'Russell Walker',...","[{'credit_id': '52fe45e2c3a368484e07460d', 'de..."
20037,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",15227,80772.0,en,Ginga tetsudô Three-Nine,"In the future, one can achieve immortality by ...",1.356131,/TDSbp7OoIQ15eZGzLiJrssIvqI.jpg,1979-08-04,...,128.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Galaxy Express 999,False,6.2,12,"[{'cast_id': 3, 'character': 'Tetsurô Hosh

In [102]:
TestData

,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,release_date,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",27205,1375666.0,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,2010-07-14,...,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075,"[{'cast_id': 1, 'character': 'Dom Cobb', 'cred...","[{'credit_id': '56e8462cc3a368408400354c', 'de..."
1,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",155,468569.0,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,2008-07-16,...,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269,"[{'cast_id': 35, 'character': 'Bruce Wayne / B...","[{'credit_id': '55a0eb4a925141296b0010f8', 'de..."
2,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,499549.0,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,2009-12-10,...,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
3,220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",24428,848228.0,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,2012-04-25,...,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000,"[{'cast_id': 46, 'character': 'Tony Stark / Ir...","[{'credit_id': '52fe4495c3a368484e02b1cf', 'de..."
4,58000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",293660,1431045.0,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,/inVq3FRqcYIRl2la8iZikYYxFNR.jpg,2016-02-09,...,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444,"[{'cast_id': 99, 'character': 'Wade Wilson / D...","[{'credit_id': '57e564fac3a3681beb0000cc', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20034,0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",41938,98093.0,en,Physical Evidence,A police officer suspended and now accused of ...,1.138586,/95JrCZlbXrfqRrbUVl6y5zmQNYp.jpg,1989-01-27,...,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If he's innocent, the truth could save him. If...",Physical Evidence,False,4.8,12,"[{'cast_id': 2, 'character': 'Joe Paris', 'cre...","[{'credit_id': '598dd8ad92514178ac015543', 'de..."
20035,0,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",52794,94713.0,en,Baxter,A white Bull Terrier named Baxter is given to ...,0.005568,/gbSiQmWfBSirJvnXmqaI1zn4n6L.jpg,1989-01-18,...,82.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,Beware of the dog who thinks.,Baxter,False,6.4,12,"[{'cast_id': 1, 'character': 'Madame Deville',...","[{'credit_id': '54359519c3a36842210002dd', 'de..."
20036,3000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",29463,89444.0,en,Krush Groove,"Russell Walker is a young, successful manager ...",1.560562,/ZccQA1MQejTGeqotMnlGOAjs1c.jpg,1985-10-25,...,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's chillin'!,Krush Groove,False,7.0,12,"[{'cast_id': 1, 'character': 'Russell Walker',...","[{'credit_id': '52fe45e2c3a368484e07460d', 'de..."
20037,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",15227,80772.0,en,Ginga tetsudô Three-Nine,"In the future, one can achieve immortality by ...",1.356131,/TDSbp7OoIQ15eZGzLiJrssIvqI.jpg,1979-08-04,...,128.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Galaxy Express 999,False,6.2,12,"[{'cast_id': 3, 'character': 'Tetsurô Hosh

In [103]:
from ast import literal_eval

features = ['cast', 'crew', 'genres']
for feature in features:
    TestData[feature] = TestData[feature].apply(literal_eval)


In [104]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [105]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [106]:
TestData['director'] = TestData['crew'].apply(get_director)

features = ['cast', 'genres']
for feature in features:
    TestData[feature] = TestData[feature].apply(get_list)

In [107]:
TestData[['title', 'cast', 'director', 'genres']]

,title,cast,director,genres
0,Inception,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",Christopher Nolan,"[Action, Thriller, Science Fiction]"
1,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger]",Christopher Nolan,"[Drama, Action, Crime]"
2,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[Action, Adventure, Fantasy]"
3,The Avengers,"[Robert Downey Jr., Chris Evans, Mark Ruffalo]",Joss Whedon,"[Science Fiction, Action, Adventure]"
4,Deadpool,"[Ryan Reynolds, Morena Baccarin, Ed Skrein]",Tim Miller,"[Action, Adventure, Comedy]"
...,...,...,...,...
20034,Physical Evidence,"[Burt Reynolds, Theresa Russell, Ned Beatty]",Michael Crichton,"[Crime, Thriller]"
20035,Baxter,"[Lise Delamare, Jean Mercure, Jacques Spiesser]",Jérôme Boivin,"[Horror, Drama, Comedy]"
20036,Krush Groove,"[Blair Underwood, Joseph Simmons, Darryl McDan...",Michael Schultz,"[Drama, Music, Romance]"
20037,Galaxy Express 999,"[Masako Nozawa, Masako Ikeda, Yôko Asagami]",Rintaro,"[Adventure, Animation, Science Fiction]"


In [108]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [109]:
features = ['cast', 'director', 'genres']

for feature in features:
    TestData[feature] = TestData[feature].apply(clean_data)

In [110]:
def create_soup(x):
    return ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
TestData['soup'] = TestData.apply(create_soup, axis=1)
TestData[['soup']]

,soup
0,leonardodicaprio josephgordon-levitt ellenpage...
1,christianbale michaelcaine heathledger christo...
2,samworthington zoesaldana sigourneyweaver jame...
3,robertdowneyjr. chrisevans markruffalo josswhe...
4,ryanreynolds morenabaccarin edskrein timmiller...
...,...
20034,burtreynolds theresarussell nedbeatty michaelc...
20035,lisedelamare jeanmercure jacquesspiesser jérôm...
20036,blairunderwood josephsimmons darrylmcdaniels m...
20037,masakonozawa masakoikeda yôkoasagami rintaro a...


In [111]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(TestData['soup'])
count_matrix.shape

(20039, 30335)

In [112]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
TestData = TestData.reset_index()
indices = pd.Series(TestData.index, index=TestData['title'])

In [113]:
 T = TestData[TestData['original_language']=='hi']
T.sort_values(by = 'popularity',ascending = False)[10:25]

,index,budget,genres,movieId,ID,original_language,original_title,overview,popularity,poster_path,...,status,tagline,title,video,vote_average,vote_count,cast,crew,director,soup
10736,10736,6400000,"[romance, drama, comedy]",118051,2168910.0,hi,Cocktail,"Gautam is an incorrigible flirt, who runs afte...",7.950488,/fTjDOuahxaalPThmfkd1eiMKMbu.jpg,...,Released,Sometimes good friends may be too good to shar...,Cocktail,False,6.2,38,"[deepikapadukone, saifalikhan, randeephooda]","[{'credit_id': '52fe4bc8c3a36847f8213af9', 'de...",homiadajania,deepikapadukone saifalikhan randeephooda homia...
1322,1322,9000000,"[drama, comedy, romance]",20453,1187043.0,hi,3 Idiots,In the tradition of “Ferris Bueller’s Day Off”...,7.675621,/wbE5SRTZFtQxgj2nIo4HJpQDk0k.jpg,...,Released,Don't BE A STUPID BE AN I.D.I.O.T.,3 Idiots,False,7.8,850,"[aamirkhan, kareenakapoor, madhavan]","[{'credit_id': '568567c692514132db019e94', 'de...",rajkumarhirani,aamirkhan kareenakapoor madhavan rajkumarhiran...
7626,7626,7100000,"[action, drama, thriller]",17501,461936.0,hi,Don : The Chase Begins Again,A huge South Asian law enforcement contingent ...,7.633636,/pRlVq9lUEuMv9Fq0t8kpPwfUT6d.jpg,...,Released,The Chase Begins Again,Don : The Chase Begins Again,False,7.3,68,"[shahrukhkhan, priyankachopra, arjunrampal]","[{'credit_id': '52fe472b9251416c750900b3', 'de...",farhanakhtar,shahrukhkhan priyankachopra arjunrampal farhan...
14396,14396,5200000,"[action, crime, romance]",403867,5460276.0,hi,Kaabil,"Rohan (Hritik roshan) is a kind, happy young m...",7.404544,/1scmyOBJkEGvb0unRCANHHH3n3S.jpg,...,Released,The Mind sees All!,Kaabil,False,8.1,23,"[hrithikroshan, yamigautam, ronitroy]","[{'credit_id': '58540476c3a368258c02652c', 'de...",sanjaygupta,hrithikroshan yamigautam ronitroy sanjaygupta ...
9005,9005,22000000,"[action, comedy, romance]",280690,2372222.0,hi,किक,An adrenaline junkie walks away from a whirlwi...,7.366331,/we4moX9pbo3DAG8VxCOcBxUsM0V.jpg,...,Released,NaN,Kick,False,6.0,51,"[salmankhan, nawazuddinsiddiqui, randeephooda]","[{'credit_id': '53bace2ac3a3685eb4007158', 'de...",sajidnadiadwala,salmankhan nawazuddinsiddiqui randeephooda saj...
13723,13723,1000000,"[thriller, romance]",392572,5165344.0,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",7.333139,/q1lrN6ZrIsOs077lQB86aPGKZRF.jpg,...,Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",tinusureshdesai,akshaykumar ileanad'cruz eshagupta tinusureshd...
5927,5927,0,"[comedy, drama]",10757,248126.0,hi,Kabhi Khushi Kabhie Gham,"Orthodox Indian, Raichand, would like his two ...",6.523888,/fqEKDhN0JVyWKQpMUPZ7We6lCLq.jpg,...,Released,It's all about loving your parents.,Kabhi Khushi Kabhie Gham,False,7.6,103,"[shahrukhkhan, kajol, amitabhbachchan]","[{'credit_id': '52fe43b09251416c7501a395', 'de...",karanjohar,shahrukhkhan kajol amitabhbachchan karanjohar ...
6284,6284,1530000,"[drama, romance]",11854,172684.0,hi,Kuch Kuch Hota Hai,Anjali is left heartbroken when her best frien...,6.441119,/gCxR77PigGO7OvewN4sAATeI363.jpg,...,Released,Something Happens,Kuch Kuch Hota Hai,False,7.5,93,"[shahrukhkhan, kajol, ranimukerji]","[{'credit_id': '52fe44939251416c75039aff', 'de...",karanjohar,shahrukhkhan kajol ranimukerji karanjohar dram...
4458,4458,14000000,"[adventure, comedy, drama]",348892,3863552.0,hi,Bajrangi Bhaijaan,A young mute girl from Pakistan loses herself ...,6.312240,/fAVCr7rfEKbQzre4ETCFXNDV6U3.jpg,...,Released,NaN,Bajrangi Bhaijaan,False,7.7,165,"[salmankhan, kareenakapoor, harshaalimalthotra]","[{'credit_id': '592602ce9251413b8a0153bf', 'de...",kabirkhan,salmankhan kareenakapoor harshaalimalthotra ka...
4069,4069,2600000,[drama],7508,986264.0,hi,Taare Zameen Par,Ishaan Awasthi is an eight-year-old whose worl...,6.093304,/t8x4j5KtI6hZRUVtPPztNBo1q5W.jpg,...,Released,Every child is special,Taare Zameen Par,False,7.7,194,"[darsheelsafary, aamirkhan, tiscachopra]","[{'cred

In [120]:
get_recommendations('The Usual Suspects', cosine_sim2)

2287            Miller's Crossing
5098           Point of No Return
4408                    Apt Pupil
16034       All Things To All Men
16792         The End of Violence
6396     And Then There Were None
19676             The Piano Tuner
138      The Silence of the Lambs
146                  The Departed
155        Leon: The Professional
Name: title, dtype: object